## **Supreme Court Transcripts Database Design**

### Contents:
 1. Finding Justices Present
 2. Date, Year
 3. Appearances
 4. Sentiment Analysis 
 5. Building the DataFrame
 

In [437]:
import pandas as pd
import regex as re
import numpy as np
import os
from os import listdir
from os.path import isfile,join

# Read in a plain text file
files = []
path = r"C:\Users\Avena Cheng\Desktop\Liberating Archives\supremecourt\textfiles\\"
for i in os.listdir(r"C:\Users\Avena Cheng\Desktop\Liberating Archives\supremecourt\textfiles"):
    if i.endswith('.txt'):
        text = open(path+i,encoding="utf8").read()
        files.append(text)

In [438]:
len(files)

1302

In [439]:
cleaned = []
for txt in files:
    clean = re.sub('\xad','',txt) # honestly not sure why this exists but ok
    clean = re.sub('\n','',clean) # no new lines
    clean = re.sub('’',"\\'",clean) #remove weird apostrophe and replace with normal apostrophe
    clean = re.sub('\\\\','',clean)
    cleaned += [clean]

In [440]:
## FIXING SPELLING ERRORS
for transcript in cleaned:
    transcript = re.sub('JUSTICE SC[A-Z]+:','JUSTICE SCALIA:',transcript)
    transcript = re.sub('JUSTICE KEN[A-Z]+','JUSTICE KENNEDY:',transcript)
    transcript = re.sub('JUSTICE [A-Z]+OUT[A-Z]+:','JUSTICE SOUTER:',transcript)
    transcript = re.sub('JUSTICE BRE[A-Z]+:','JUSTICE BREYER:',transcript)
    transcript = re.sub('JUSTICE GIN[A-Z]+:','JUSTICE GINSBURG',transcript)
    transcript = re.sub('JUSTICE [A-Z]+QUIST:','JUSTICE REHNQUIST:',transcript)
    transcript = re.sub('JUSTICE SOTO[A-Z]+:','JUSTICE SOTOMAYOR:',transcript)
    transcript = re.sub('JUSTICE ROB[A-Z]+:','JUSTICE ROBERTS:',transcript)
    transcript = re.sub('JUSTICE STEV[A-Z]+:','JUSTICE STEVENS:',transcript)

### **Finding Justices Present**

In [441]:
def unique(lst):
    uni = []
    for i in lst:
        if i not in uni:
            uni += [i]
    return uni

def case_no(txt):
    return unique(re.findall('No.\s*\d+[-]*\d+',txt))

In [442]:
def justices(txt):
    j = re.findall('JUSTICE[A-Z\s]+:',txt)
    justice = sorted(unique(j))
    cleaned_list = [justice[i][:-1] for i in range(len(justice))]
    return cleaned_list

In [443]:
justices(cleaned[0])

['JUSTICE ALITO',
 'JUSTICE BREYER',
 'JUSTICE GINSBURG',
 'JUSTICE KENNEDY',
 'JUSTICE ROBERTS',
 'JUSTICE SCALIA',
 'JUSTICE SOUTER',
 'JUSTICE STEVENS']

In [444]:
#def justices(texts):
 #   d = {}
  #  for txt in texts:
   #     num = case_no(txt)[0]
    #    j = re.findall('JUSTICE[A-Z\s]+:',txt)
     #   justice = sorted(unique(j))
      #  cleaned_list = [justice[i][:-1] for i in range(len(justice))]
       # d[num] = cleaned_list
    #return d

## **Date, Year**

In [445]:
def date(text):
    return re.findall('\W\w+,*\s+\d+\w*\w*,\s+\d{4}',text)

In [446]:
def year(text):
    return re.findall('\d\d\d\d',date(text))

## **Appearances**

In [447]:
def appearances(text):
    app = re.findall('APPEARANCES:[\s\S]*?Reporting',text)[0]
    app = re.findall('[\s\S]*?;[\s\S]*?\.',app)
    app = [re.sub('\d','',app[i]) for i in range(len(app))]
    remove_appearance = re.sub('APPEARANCES:\s','',app[0])
    app[0] = remove_appearance
    app = [re.sub('[\s\s]+',' ',app[i]) for i in range(len(app))]
    return app

In [448]:
#for i in cleaned:
 #   app = appearances(i)
  #  for j in app:
   #     print(j)
    #print('-----------------------')

## **Sentiment Analysis**

**Checkpoint 10/22**

* need to work on finding the regex pattern between speakers
* General pattern: "SPEAKER: anything they say until the next SPEAKER:"

### STEPS:
1. Extract the sentences from each speaker.
2. Develop a function (actually there's one written in the github link below that you could model yours from; it's very good)
3. Test it on various transcripts to ensure it's generalized

### Note to Amal: Please look at these slides for performing sentiment analysis. They are from my IEOR class, and these techniques are very useful. Please let me know if you would like to go over it together.

https://github.com/ikhlaqsidhu/data-x/blob/master/07a-tools-nlp-sentiment_add_missing_si/notebook-nlp-sentiment-analysis-imdb-afo_v2.ipynb

## OTHER NOTES: you can't use the function below on cases where the justices' names are replaced with "QUESTION"

In [449]:
reporting_pattern = '[A-Z][a-z]+\sReporting'
REPORTING_pattern = '[A-Z]+\sREPORTING'
contents_pattern = 'CONTENTS'
CONTENTS_pattern = 'C O N T E N T S'

In [450]:
## I was trying to extract the speakers from each text. 
all_justices =['JUSTICE GORSUCH','JUSTICE PHILLIPS','JUSTICE SCALIA','JUSTICE ROBERTS','JUSTICE KAVANAUGH',
               'JUSTICE GINSBURG','JUSTICE BREYER','JUSTICE KENNEDY','JUSTICE ALITO','JUSTICE STEVENS',
               'JUSTICE KAGAN','JUSTICE REHNQUIST','JUSTICE THOMAS','JUSTICE SOTOMAYOR','JUSTICE SOUTER','JUSTICE BENNETT']

def dialogue(text,person):
    pattern = re.escape(person)+r":[\s\S]+?[A-Z]+\s*?:"
    sents = re.findall(pattern,text) #regex pattern to find all instances
    sents = [re.sub('\d','',i) for i in sents] # cleaning transcript
    sents = [re.sub('[\s\s]+',' ',i) for i in sents] #cleaning transcript
    sents = [re.sub('Company Official',' ',i) for i in sents if len(re.sub('Company Official',' ',i)) != 0]
    for i in range(len(sents)):
        report1 = re.findall(reporting_pattern,sents[i])
        report2 = re.findall(REPORTING_pattern,sents[i])
        content1 = re.findall(contents_pattern,sents[i])
        content2 = re.findall(CONTENTS_pattern,sents[i])
        if len(report1) > 0:
            sents[i]=re.sub(report1[0],'',sents[i])
        elif len(report2) > 0:
            sents[i]=re.sub(report2[0],'',sents[i])
        elif len(content1) > 0:
            sents[i]=re.sub(content1[0],'',sents[i])
        elif len(content2) > 0:
            sents[i]=re.sub(content2[0],'',sents[i])
        else:
            continue
    sents = [re.sub('Official - Subject to Final Review','',i) for i in sents if len(re.sub('Official - Subject to Final Review','',i))!= 0]
    sents = [re.sub('- Subject to Final Review','',i) for i in sents]
    sents = [re.sub('JUSTICE [A-Z]+:','',i) for i in sents]
    sents = [re.sub('CHIEF|Page|JUSTICE','',i) for i in sents]
    sents = [re.sub('th Street, NW Suite  Company Washington, DC --FOR-DEPO','',i) for i in sents]
    sents = [re.sub('--FOR-DEPO','',i) for i in sents]
    sents = [re.sub('[A-Z\s*\.*]+:','',i) for i in sents]
    return sents


In [451]:
dialogue(cleaned[1123],'REHNQUIST')

[" We'll hear argument next in No. -, TRW, Inc. v. Adelaide Andrews. Mr. Nager"]

In [452]:
case_1 = []
speech = []
for j in all_justices:
    for i in range(100):
        text =  dialogue(cleaned[i],j)
        speech += [text]
        

In [453]:
len(all_justices)

16

In [454]:
#speech

# **Building the DataFrame**

#### Justices!
`#print(cleaned[4])`

If we run the above ...

Notice that instead of having the justices' name, it just says "QUESTION". This is usually the case for when it appears there is only one justice present. It seems to be only for Justice Rehnquist.

In [455]:
## Testing do not use
trial = [justices(i) for i in cleaned]
every_justice = []
for i in trial:
    every_justice += i
uniques = unique(every_justice)
for i in range(len(uniques)):
    uniques[i] = re.sub('JUSTICE','',uniques[i])
    uniques[i] = re.sub('\s+','',uniques[i])
#unique(uniques)

In [456]:
justice = [justices(i) for i in cleaned]
for lst in justice:
    for word in range(len(lst)):
        #lst[word] = re.sub('JUSTICE','',lst[word])
        #lst[word] = re.sub('\s+','',lst[word])
        if 'SC' in lst[word]:
            lst[word] = re.sub(lst[word],'JUSTICE SCALIA',lst[word])
        if 'KEN' in lst[word]:
            lst[word] = re.sub(lst[word],'JUSTICE KENNEDY',lst[word])
        if 'TER' in lst[word]:
            lst[word] = re.sub(lst[word],'JUSTICE SOUTER',lst[word])
        if 'BRE' in lst[word]:
            lst[word] = re.sub(lst[word],'JUSTICE BREYER',lst[word])
        if 'GIN' in lst[word]:
            lst[word] = re.sub(lst[word],'JUSTICE GINSBURG',lst[word])
        if 'QUIST' in lst[word]:
            lst[word] = re.sub(lst[word],'JUSTICE REHNQUIST',lst[word])
        if 'SOTO' in lst[word]:
            lst[word] = re.sub(lst[word],'JUSTICE SOTOMAYOR',lst[word])
        if 'ROB' in lst[word]:
            lst[word] = re.sub(lst[word],'JUSTICE ROBERTS',lst[word])
        if 'STEVE' in lst[word]:
            lst[word] = re.sub(lst[word],'JUSTICE STEVENS',lst[word])

In [457]:
#for i in range(len(justice)):
 #   justice[i] = re.sub('JUSTICE','',justice[i])

#### Dates!

In [458]:
date(cleaned[0])[0]

' December 1, 2008'

In [459]:
dates = []
for i in cleaned:
    dates += [date(i)[0]]

In [460]:
len(dates)

1302

#### Year!

In [461]:
years = []
for i in dates:
    years += re.findall('\d\d\d\d',i)

In [462]:
len(years)

1302

#### Case Numbers! (Needs improvement)

In [466]:
case_no(cleaned[0])

['No. 07-581']

In [467]:
cases = []
for i in cleaned:
    cases += [case_no(i)[0]]

In [468]:
len(cleaned)

1302

#### Appearances!

In [469]:
# don't use this one
def appearances(text):
    app = re.findall('APPEARANCES:[\s\S]*?Reporting.*?',text)
    if len(app) == 0:
        app = re.findall('APPEARANCES:[\s\S]*?REPORTING.*?',text)
        if len(app) == 0:
            app = re.findall('APPEARANCES:[\s\S]*?C O N T E N T S.*?',text)
            if len(app) == 0:
                app = re.findall('APPEARANCES:[\s\S]*?CONTENTS.*?',text)
    app = app[0]
    app = re.findall('[\s\S]*?;[\s\S]*?\.',app)
    app = [re.sub('\d','',app[i]) for i in range(len(app))]
    remove_appearance = re.sub('APPEARANCES:\s','',app[0])
    app[0] = remove_appearance
    app = [re.sub('[\s\s]+',' ',app[i]) for i in range(len(app))]
    return app

In [470]:
def diff(text):
    app = re.findall('APPEARANCES:*[\s\S]*?Reporting.*?',text)
    if len(app) == 0 or len(app[0])>1000:
        app = re.findall('APPEARANCES:[\s\S]*?REPORTING.*?',text)
        if len(app) == 0 or len(app[0])>1000:
            app = re.findall('APPEARANCES:[\s\S]*?C O N T E N T S.*?',text)
            if len(app) == 0 or len(app[0])>1000:
                app = re.findall('APPEARANCES:[\s\S]*?CONTENTS.*?',text)
                if len(app) == 0 or len(app[0])>1000:
                    app = re.findall('APPEARANCES:[\s\S]*?P R O C E E D I N G S.*?',text)
    app = app[0]
    app = re.sub('\d','',app)
    remove_appearance = re.sub('APPEARANCES:*\s','',app)
    app = re.sub('[\s\s]+',' ',remove_appearance)
    return app

In [471]:
ugh = []
c = 1
for i in range(len(cleaned)):
    #print(c)
    try:
        ugh+=[diff(cleaned[i])]
        #print('')
    except:
        ugh += [i]
        continue
    c+=1

In [472]:
for i in ugh:
    if type(i)==int:
        print(i)

1023


In [473]:
ugh[1023]

1023

In [474]:
for i in range(len(ugh)):
    if i == 1023:
        ugh[i] = 1023
    else:
        ugh[i] = re.sub('in No. -','',ugh[i])

In [475]:
ugh[0]

'PAUL SALVATORE, ESQ., New York, N.Y.; on behalf of the Petitioners. DAVID C. FREDERICK, ESQ., Washington, D.C.; on behalf of the Respondents. CURTIS E. GANNON, ESQ., Assistant to the Solicitor General, Department of Justice, Washington, D.C.; on behalf of the United States, as amicus curiae, supporting the Respondents. Alderson Reporting'

In [476]:
re.findall(reporting_pattern,cleaned[0])[0]

'Alderson Reporting'

In [477]:
re.sub(re.findall(reporting_pattern,ugh[0])[0],'',ugh[0])

'PAUL SALVATORE, ESQ., New York, N.Y.; on behalf of the Petitioners. DAVID C. FREDERICK, ESQ., Washington, D.C.; on behalf of the Respondents. CURTIS E. GANNON, ESQ., Assistant to the Solicitor General, Department of Justice, Washington, D.C.; on behalf of the United States, as amicus curiae, supporting the Respondents. '

In [478]:
for i in range(len(ugh)):
    if i == 1023:
        ugh[i] == 'JASON D. HAWKINS, ESQ., Assistant Federal Public Defender, Dallas, Texas; for Petitioner. WILLIAM M. JAY, ESQ., Assistant to the Solicitor General, Department of Justice, Washington, D.C.; for  Respondent, in support of Petitioner. EVAN A. YOUNG, ESQ., Austin, Texas; for amicus curiae, in support of the judgment below; appointed by this Court.'
    else:
        report1 = re.findall(reporting_pattern,ugh[i])
        report2 = re.findall(REPORTING_pattern,ugh[i])
        content1 = re.findall(contents_pattern,ugh[i])
        content2 = re.findall(CONTENTS_pattern,ugh[i])
        if len(report1) > 0:
            ugh[i]=re.sub(report1[0],'',ugh[i])
        elif len(report2) > 0:
            ugh[i]=re.sub(report2[0],'',ugh[i])
        elif len(content1) > 0:
            ugh[i]=re.sub(content1[0],'',ugh[i])
        else:
            ugh[i]=re.sub(content2[0],'',ugh[i])

In [479]:
bad = 'JASON D. HAWKINS, ESQ., Assistant Federal Public Defender, Dallas, Texas; for Petitioner. WILLIAM M. JAY, ESQ., Assistant to the Solicitor General, Department of Justice, Washington, D.C.; for  Respondent, in support of Petitioner. EVAN A. YOUNG, ESQ., Austin, Texas; for amicus curiae, in support of the judgment below; appointed by this Court.'

In [480]:
no_55 = re.sub("\d", "",cleaned[55])
no_295 = re.sub("\d","",cleaned[295])
app_55 = "THOMAS C. GOLDSTEIN, ESQ., Washington, D.C.; on behalf of Petitioners. DONALD B. VERRILLI, JR., ESQ., Solicitor General, Department of Justice, Washington, D.C.; on behalf of Respondents."
app_295 = "APPEARANCES: Alderson Reporting Company Official - Subject to Final Review TIMOTHY S. BISHOP, ESQ., Chicago, Illinois; on behalf of Petitioners. MALCOLM L. STEWART, ESQ., Deputy Solicitor General, Department of Justice, Washington, D.C.; for United States, as amicus curiae, in support of Petitioners. JEFFREY L. FISHER, ESQ., Stanford, California; on behalf of Respondents."
app_1023 = 'JASON D. HAWKINS, ESQ., Assistant Federal Public Defender, Dallas, Texas; for Petitioner. WILLIAM M. JAY, ESQ., Assistant to the Solicitor General, Department of Justice, Washington, D.C.; for  Respondent, in support of Petitioner. EVAN A. YOUNG, ESQ., Austin, Texas; for amicus curiae, in support of the judgment below; appointed by this Court.'

In [481]:
ugh[55] = app_55
ugh[295] = app_295
ugh[1023] = app_1023

#### People!

In [482]:
people = []
for i in range(len(ugh)):
    p = re.findall('[A-Z]+\s[A-Z\.\s]*[\'*A-Za-z\s*]+',ugh[i])
    people += [p]
    
for i in range(len(people)):
    people[i] = ", ".join(people[i])

In [483]:
anns = []
for i in range(len(people)):
    lst = people[i]
    if "DABNEY" in lst:
        anns += [i]

In [484]:
for i in anns:
    print(data.iloc[i]['Title'])
    print('')

Already, LLC v. Nike, Inc.

Holmes Group, Inc. v. Vornado Air Circulation Systems, Inc.

KSR International Co. v. Teleflex, Inc.

TC Heartland LLC v. Kraft Foods Group Brands LLC



In [485]:
data.iloc[1252]['Appearances']

"MICHAEL B. KIMBERLY, ESQ., Washington, D.C.; on behalf of the Petitioner. RANDALL E. RAVITZ, ESQ., Assistant Attorney General, Boston, Mass.; on behalf of the Respondent. ANN O'CONNELL, ESQ., Assistant to the Solicitor General, Department of Justice, Washington, D.C.; for United States, as amicus curiae, supporting the Respondent. "

In [486]:
#for i in range(len(people)):
 #   need_to_fix = re.findall()

In [487]:
data.iloc[825]

Title                                            Nevada v. Hicks
Case_ID                                                      826
Case_No                                              No. 99-1994
Date                                              March 21, 2001
Year                                                        2001
Justices                                     [JUSTICE REHNQUIST]
Appearances    C. WAYNE HOWLE, ESQ., Carson City, Nevada; on ...
People                WAYNE HOWLE, JAMES ANAYA, BARBARA MCDOWELL
Name: 825, dtype: object

#### Title!

In [488]:
title = [f for f in listdir(r"C:\Users\Avena Cheng\Desktop\Liberating Archives\supremecourt\textfiles") if isfile(join(r"C:\Users\Avena Cheng\Desktop\Liberating Archives\supremecourt\textfiles", f))]
for i in range(len(title)):
    title[i] = re.sub('.pdf.txt','',title[i])
title.remove('DatabaseDesign1.ipynb')

### ACTUAL DATAFRAME

** Note: Some cases only have one justice, Rehnquist, but this is actually because the rest of the questions appear as "QUESTION:" rather than the justices' name.

In [489]:
#make sure they're all the same length
print(len(justice),
len(title),
len(ugh),
len(people),
len(years),
len(cases))

1302 1302 1302 1302 1302 1302


In [490]:
data = pd.DataFrame({'Title':title,'Case_ID':range(1,1303),'Case_No': cases,'Date':dates,'Year':years,'Justices':justice,'Appearances':ugh,'People':people})#'Justices':justice,'Appearances':people})

In [491]:
data.head()

,Title,Case_ID,Case_No,Date,Year,Justices,Appearances,People
0,14 Penn Plaza LLC v. Pyett,1,No. 07-581,"December 1, 2008",2008,"[JUSTICE ALITO, JUSTICE BREYER, JUSTICE GINSBU...","PAUL SALVATORE, ESQ., New York, N.Y.; on behal...","PAUL SALVATORE, DAVID C. FREDERICK, CURTIS E. ..."
1,Abbott v. Perez,2,No. 17-586,"April 24, 2018",2018,"[JUSTICE ALITO, JUSTICE BREYER, JUSTICE GINSBU...","SCOTT A. KELLER, Solicitor General of Texas, ...","SCOTT A. KELLER, EDWIN S. KNEEDLER, MAX RENEA ..."
2,Abbott v. United States,3,No. 09-479,"October 4, 2010",2010,"[JUSTICE ALITO, JUSTICE BREYER, JUSTICE GINSBU...","DAVID L. HORAN, ESQ., Dallas, Texas; on behalf...","DAVID L. HORAN, JAMES E. RYAN"
3,Abdul-Kabir v. Quarterman,4,No. 05-11284,"January 17, 2006",2006,"[JUSTICE ALITO, JUSTICE BREYER, JUSTICE GINSBU...","ROBERT C. OWEN, ESQ., Austin, Tex.; on behalf...","ROBERT C. OWEN, EDWARD L. MARSHALL"
4,Abdur_Rahman v. Bell,5,No. 01-9094,"November 6, 2002",2002,[JUSTICE REHNQUIST],"JAMES S. LIEBMAN, ESQ., New York, New York; on...","JAMES S. LIEBMAN, PAUL G. SUMMERS, PAUL J. ZID..."


In [492]:
data.to_csv(r"C:\Users\Avena Cheng\Desktop\Liberating Archives\data.csv")

## Bills/Acts

** credits to Yuhan and Amal**

In [497]:
import regex as re
import numpy
import os

def unique(texts):
    return list(set(texts))

#return a set of nonunique bill mentioned in the speech.
def bill(texts):
    no_num = re.sub('\d+','',texts)
    no_n = re.sub('\n','',no_num)
    evenly_spaced = re.sub('\s+',' ',no_n)
    b= re.findall("[A-Z][a-z]+\s[A-Z][a-z]+\sAct", evenly_spaced)
    return b
#return a set of unique bill mentioned in the speech.
def unibill(texts):
    return unique(bill(texts))
#return the name and times it appears.
def count(texts):
    bills = bill(texts)
    ret = []
    for i in range(0,len(bills)):
        count = bills.count(bills[i])
        ret.append (str(bills[i]) + " " + ":" + ' '+str(count))
    return unique(ret)


In [498]:
unibill(cleaned[0])
count(cleaned[0])

['Labor Standards Act : 1', 'Labor Relations Act : 2']

In [499]:
bills_mentioned = []
for i in range(len(cleaned)):
    bills_mentioned += [count(cleaned[i])]

In [500]:
len(bills_mentioned)

1302

In [501]:
data['Bills Mentioned'] = bills_mentioned

In [508]:
bills_db = data[['Title','Case_No','Date','Bills Mentioned']]

In [510]:
#bills_db.to_csv('bills_db.csv')

## Keywords/TF-IDF

In [511]:
import nltk

# nltk.download('all')
from nltk.tokenize import sent_tokenize # tokenizes sentences
import re

from nltk import word_tokenize
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet

eng_stopwords = stopwords.words('english')

In [512]:
def text_cleaner(text,stem=False):
    '''
    1. Use regex to remove all special characters (only keep letters)
    2. Make strings to lower case and tokenize / word split reviews
    3. Remove English stopwords
    4. Rejoin to one string
    '''
    #3. Remove punctuation
    text = re.sub("[^a-zA-Z]", " ",text)
    #4. Tokenize into words (all lower case)
    text = text.lower()
    #5. Remove stopwords
    eng_stopwords = set(stopwords.words("english"))

    words = word_tokenize(text)
    words = [w for w in words if w not in eng_stopwords]
    return(words)

In [513]:
def term_frequency(document, term):
    words = text_cleaner(document)
    number_of_words = len(words)
    print("Total number of words in document 1 is", number_of_words)

    frequency_of_word = sum([1 for word in words if word.lower() == term])
    print("Frequency of the word", term, "is", frequency_of_word)

    term_frequency = frequency_of_word / number_of_words
    return term_frequency

In [514]:
#text_cleaner(cleaned[0])

## SPEECHES DATAFRAME 
### Just justices right now. we can add other people later.

In [61]:
speech = []
for j in all_justices:
    for i in range(1):
        text =  dialogue(cleaned[i],j)
        speech += [text]
        

In [515]:
len(speech)

1600

In [516]:
data.head()

,Title,Case_ID,Case_No,Date,Year,Justices,Appearances,People,Bills Mentioned
0,14 Penn Plaza LLC v. Pyett,1,No. 07-581,"December 1, 2008",2008,"[JUSTICE ALITO, JUSTICE BREYER, JUSTICE GINSBU...","PAUL SALVATORE, ESQ., New York, N.Y.; on behal...","PAUL SALVATORE, DAVID C. FREDERICK, CURTIS E. ...","[Labor Standards Act : 1, Labor Relations Act ..."
1,Abbott v. Perez,2,No. 17-586,"April 24, 2018",2018,"[JUSTICE ALITO, JUSTICE BREYER, JUSTICE GINSBU...","SCOTT A. KELLER, Solicitor General of Texas, ...","SCOTT A. KELLER, EDWIN S. KNEEDLER, MAX RENEA ...",[Voting Rights Act : 3]
2,Abbott v. United States,3,No. 09-479,"October 4, 2010",2010,"[JUSTICE ALITO, JUSTICE BREYER, JUSTICE GINSBU...","DAVID L. HORAN, ESQ., Dallas, Texas; on behalf...","DAVID L. HORAN, JAMES E. RYAN",[Career Criminal Act : 3]
3,Abdul-Kabir v. Quarterman,4,No. 05-11284,"January 17, 2006",2006,"[JUSTICE ALITO, JUSTICE BREYER, JUSTICE GINSBU...","ROBERT C. OWEN, ESQ., Austin, Tex.; on behalf...","ROBERT C. OWEN, EDWARD L. MARSHALL",[]
4,Abdur_Rahman v. Bell,5,No. 01-9094,"November 6, 2002",2002,[JUSTICE REHNQUIST],"JAMES S. LIEBMAN, ESQ., New York, New York; on...","JAMES S. LIEBMAN, PAUL G. SUMMERS, PAUL J. ZID...",[]


In [978]:
speech_df = pd.DataFrame({'Name':[],'Case':[],'Speech':[]})
for i in range(1302):
    title = data['Title'].iloc[i]
    judges = data['Justices'].iloc[i]
    case_id = data['Case_ID'].iloc[i]
    for j in judges:
        words = dialogue(cleaned[i],j)
        words = ".".join(words)
        speech_df = speech_df.append(pd.DataFrame({'Name':[j],'Case':[title],'Case_ID':[case_id],'Speech':[words]}))

C:\Anaconda3\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


In [980]:
speech_df['Case_ID'] = speech_df['Case_ID'].astype(int)

In [981]:
len(speech_df)

7852

In [982]:
speech_df.head()

,Case,Case_ID,Name,Speech
0,14 Penn Plaza LLC v. Pyett,1,JUSTICE ALITO,Has any court decided this issue of the inter...
0,14 Penn Plaza LLC v. Pyett,1,JUSTICE BREYER,I ask you -- let me ask you a naive question ...
0,14 Penn Plaza LLC v. Pyett,1,JUSTICE GINSBURG,But here it wasn't. . Here we are dealing...
0,14 Penn Plaza LLC v. Pyett,1,JUSTICE KENNEDY,Yes. What -- what - . I don't -- I don't wish...
0,14 Penn Plaza LLC v. Pyett,1,JUSTICE ROBERTS,"We'll hear argument next in Case -, Penn Plaz..."


### This table contains everything that each justice says, in every court case.
Remember that we will have missing values because in some cases, the rest of the justices are labeled as "QUESTION:" so we don't actually know who is speaking.

In [522]:
speech_df.head()

,Case,Case_ID,Dialogue,Name
0,14 Penn Plaza LLC v. Pyett,1,Has any court decided this issue of the inter...,JUSTICE ALITO
0,14 Penn Plaza LLC v. Pyett,1,I ask you -- let me ask you a naive question ...,JUSTICE BREYER
0,14 Penn Plaza LLC v. Pyett,1,But here it wasn't. . Here we are dealing...,JUSTICE GINSBURG
0,14 Penn Plaza LLC v. Pyett,1,Yes. What -- what - . I don't -- I don't wish...,JUSTICE KENNEDY
0,14 Penn Plaza LLC v. Pyett,1,"We'll hear argument next in Case -, Penn Plaz...",JUSTICE ROBERTS


In [523]:
#data[['Justices']]

In [524]:
#cleaned[1123]

In [525]:
#speech_df.to_csv('speeched_db.csv')

In [526]:
speech_df.isnull().any()

Case        False
Case_ID     False
Dialogue    False
Name        False
dtype: bool

### Working on everyone else's speeches

In [934]:
people_df = pd.read_csv(r"C:\Users\Avena Cheng\Desktop\Liberating Archives\people.csv")
people_df = people_df.drop(1878).drop(1435).drop(1815)
people_df = people_df.reset_index().drop('index',axis=1).drop('Unnamed: 0',axis=1)
for i in people_df.loc[people_df['Name'] == 'DOUGLAS HALLWARD'].index:
    people_df['Name'][i] == "DOUGLAS HALLWARD-DRIEMEIER"
for i in people_df.loc[people_df['Name'] == "BONNIE I. ROBIN"].index:
    people_df['Name'][i] = 'BONNIE I. ROBIN-VERGEER'

In [935]:
len(people_df)

3272

#### Approach:
1. Find all the last names of people
2. using regex, perform the same for loop as your did with justices, but substitute with people. 
3. good luck.

In [958]:
last_names = []
for i in range(3272):
    name = people_df['Name'][i]
    if 'ESQ' in name:
        name = name.strip('ESQ')
    if len(re.findall('\.+',name)) > 0: # For those with middle names
        names = re.findall("[A-Z\'*a-z*]+",name)
        if len(re.findall('[a-z]+',name))>0: # For those with a name like "McKINSEY" or "DeLONE"
            last_names += [names[len(names)-1]]
        else:
            last_names += [names[len(names)-1]]
    else: # For those w/o middle names
        names = re.findall('[A-Z\'*a-z*]+',name)
        if len(re.findall('[a-z]+',name))>0:
            last_names += [names[len(names)-1]]
        else:
            last_names += [names[len(names)-1]]

In [966]:
# Fixing spelling mistakes
last_names[2200] = 'LABELLE'
last_names[554] = 'GERSHENGORN'
last_names[2809] = 'GOLDENBERG'
last_names[1503] = 'McALLISTER'
last_names[2601] = 'ROSENCRANZ'
indices_hallward = [i for i, x in enumerate(last_names) if x == "HALLWARD"]
indices_robin = [i for i, x in enumerate(last_names) if x == "ROBIN"]

for i in indices_hallward:
    last_names[i] = 'HALLWARD-DRIEMEIER'
for i in indices_robin:
    last_names[i] = 'ROBIN-VERGEER'

In [968]:
people_df.head()

,Name,Case,Last
0,PAUL SALVATORE,14 Penn Plaza LLC v. Pyett,SALVATORE
1,DAVID C. FREDERICK,14 Penn Plaza LLC v. Pyett,FREDERICK
2,CURTIS E. GANNON,14 Penn Plaza LLC v. Pyett,GANNON
3,SCOTT A. KELLER,Abbott v. Perez,KELLER
4,EDWIN S. KNEEDLER,Abbott v. Perez,KNEEDLER


In [969]:
people_df['Last'] = last_names
case_table = data[['Title','Case_ID']]
case_table = case_table.rename({'Title':'Case'},axis=1)
speech_table = pd.merge(people_df,case_table)

In [970]:
len(speech_table)

3272

In [971]:
#takes about 2 minutes to run
speeches=[]
for i in range(3272):
    caseid = speech_table['Case_ID'][i]-1
    d = dialogue(cleaned[caseid],speech_table['Last'][i])
    d = ".".join(d)
    if len(d) > 0:
        speeches += [d]
    else:
        speeches += ['Null']

In [972]:
speech_table['Speech']=speeches

In [977]:
speech_df.head()

,Case,Case_ID,Dialogue,Name
0,14 Penn Plaza LLC v. Pyett,1,Has any court decided this issue of the inter...,JUSTICE ALITO
0,14 Penn Plaza LLC v. Pyett,1,I ask you -- let me ask you a naive question ...,JUSTICE BREYER
0,14 Penn Plaza LLC v. Pyett,1,But here it wasn't. . Here we are dealing...,JUSTICE GINSBURG
0,14 Penn Plaza LLC v. Pyett,1,Yes. What -- what - . I don't -- I don't wish...,JUSTICE KENNEDY
0,14 Penn Plaza LLC v. Pyett,1,"We'll hear argument next in Case -, Penn Plaz...",JUSTICE ROBERTS


In [996]:
len(speech_table)

3272

In [1002]:
everyone_speech = speech_table.append(speech_df).drop('Last',axis=1)

C:\Anaconda3\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


In [1005]:
len(everyone_speech)

11124

#### Exploring Null Cases

So far I've discovered why there are null cases:
1. problems with my "last name" extraction (i.e. some people have hyphens, such as HALLWARD-); **fixed**
2. SPELLING ERRORS ON THE REPORTING COMPANY'S BEHALF  **fixed**
 * Case 221: it should be GERSHERNGORN not GERSHENGORN
 * Case 1032: ROSENKRANZ vs ROSENCRANZ
 * Case 1113: GOLDENBERG vs GOLENDBERG
3. Slight errors in my function for finding "appearances" -- these cases should simply be deleted. **fixed**
 * Case 744: diff(cleaned[743]) will return GENERAL because of the regex pattern)
 * Case 569: AFSCME Council
 * Case 719: Services
4. Case 870: LaBELLE is null because whenever she speaks, her name is fully capitalized despited the lowercase 'a'. **fixed**

In [974]:
speech_table.loc[speech_table['Speech'] == 'Null']

,Name,Case,Last,Case_ID,Speech


## Convert speech to sql



In [89]:
speech_df = speech_df.reset_index().drop('index',axis=1).head()

In [1006]:
import sqlite3
con = sqlite3.connect("database.db")

everyone_speech.to_sql("speech", con, if_exists="replace")


In [1007]:
everyone_speech.head()

,Case,Case_ID,Name,Speech
0,14 Penn Plaza LLC v. Pyett,1,PAUL SALVATORE,"Thank you, Mr. Chief Justice, and may it plea..."
1,14 Penn Plaza LLC v. Pyett,1,DAVID C. FREDERICK,"Thank you, Mr. Chief Justice, and may it plea..."
2,14 Penn Plaza LLC v. Pyett,1,CURTIS E. GANNON,"Mr. Chief Justice, and may it please the Cour..."
3,Abbott v. Perez,2,SCOTT A. KELLER,"Thank you, Mr. Chief Justice, and may it plea..."
4,Abbott v. Perez,2,EDWIN S. KNEEDLER,"Mr. Chief Justice, and may it please the Cour..."
